In [ ]:
# May have to remove / comment out if training info not present
%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [4]:
# IMPORTS
import numpy as np
import pandas as pd
import os
import librosa
from datetime import datetime 

# PROCESSING
from sklearn.preprocessing import LabelEncoder

# MODEL
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical # np utils
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, SpatialDropout2D, PReLU, LeakyReLU, BatchNormalization
from tensorflow.keras.layers import Convolution2D, Conv2D, GlobalAveragePooling2D, MaxPooling2D, SpatialDropout2D
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import np_utils
from sklearn import metrics 

# TRAINING
from tensorflow.keras.callbacks import ModelCheckpoint 

print("Finished Importing")


Finished Importing


In [8]:
num_rows = 40
num_columns = 174
num_channels = 1        

# Extract Features
def extract_features(file_name):
    max_pad_len = 174
    try:
        # Loading audio file
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        # Extracting MFCCs
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

        print(mfccs.shape)
        # If we have to many features, cut them, not needed
        if (mfccs.shape[1] <= max_pad_len):
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else: # Limit the number of features we use to 174
            mfccs = mfccs[:,:174]
        
    except Exception as e:
        print("Error parsing: ", file_name)
        return None 
     
    return mfccs

def print_prediction(file_name, model):
    features = extract_features(file_name) 
    features = features.reshape(1, num_rows, num_columns, num_channels)

    predicted_vector = model.predict_classes(features)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("Predicted class is:", predicted_class[0], '\n') 

    proba_vector = model.predict(features) 
    predicted_proba = proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [9]:
from tensorflow.keras.models import Sequential, load_model
model_test = load_model('saved_models/new_model_4_class')



#score = model_test.evaluate(x_test, y_test, verbose=0)
#print("Verified Testing Accuracy: ", score[1])

In [7]:
model_test = load_model('saved_models/new_model_4_class')


filename = 'audio/drums.wav'

print_prediction(filename, model_test) 

(40, 317)
/home/vithuran/anaconda3/envs/tf-gpu/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


NameError: name 'le' is not defined